### Transform premise DataFrame

In [1]:
# # Create a filtered dataframe from specific columns
# premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
# premise_transformed= premise_df[premise_cols].copy()

# # Rename the column headers
# premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
#                                                           "Premises Name": "premise_name",
#                                                           "County ID Code": "county_id"})

# # Clean the data by dropping duplicates and setting the index
# premise_transformed.drop_duplicates("id", inplace=True)
# premise_transformed.set_index("id", inplace=True)

# premise_transformed.head()

### Transform county DataFrame

In [2]:
# county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
# county_transformed = county_df[county_cols].copy()

# # Rename the column headers
# county_transformed = county_transformed.rename(columns={"ID": "id",
#                                                          "County Name (Licensee)": "county_name",
#                                                          "License Count": "license_count",
#                                                          "County ID Code": "county_id"})

# # Set index
# county_transformed.set_index("id", inplace=True)

# county_transformed.head()

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from config import sql_key

### Extract CSVs into DataFrames

In [4]:
local_govt_areas_file = "01_Inputs/lga_id_table.csv"
local_govt_areas_df = pd.read_csv(local_govt_areas_file)
local_govt_areas_df.head()

,lga,lga_id
0,banyule,1
1,darebin,2
2,whittlesea,3
3,nillumbik,4
4,bayside,5


In [18]:
local_govt_areas_df= local_govt_areas_df.rename(columns={"lga":"lga_name"})
local_govt_areas_df = local_govt_areas_df[["lga_id","lga_name"]]
local_govt_areas_df

,lga_id,lga_name
0,1,banyule
1,2,darebin
2,3,whittlesea
3,4,nillumbik
4,5,bayside
5,6,glen eira
6,7,kingston
7,8,boroondara
8,9,whitehorse
9,10,stonnington


In [5]:
suburbs_file = "01_Inputs/suburbs_postcode_lat_lon.csv"
suburbs_df = pd.read_csv(suburbs_file)
suburbs_df.head()

,postcode,suburb,lat,lon
0,3067,abbotsford,-37.801781,144.998752
1,3040,aberfeldie,-37.756690,144.896259
2,3042,airport west,-37.711698,144.887037
3,3021,albanvale,-37.745934,144.770027
4,3206,albert park,-37.840705,144.955710


In [23]:
suburbs_df = suburbs_df.drop_duplicates(subset="suburb")

In [24]:
suburbs_df

,postcode,suburb,lat,lon
0,3067,abbotsford,-37.801781,144.998752
1,3040,aberfeldie,-37.756690,144.896259
2,3042,airport west,-37.711698,144.887037
3,3021,albanvale,-37.745934,144.770027
4,3206,albert park,-37.840705,144.955710
...,...,...,...,...
311,3016,williamstown,-37.856902,144.897698
312,3016,williamstown north,-37.857681,144.887041
313,3181,windsor,-37.856159,144.992200
314,3085,yallambie,-37.727482,145.102309


In [6]:
crime_types_file = "01_Inputs/crime_type.csv"
crime_types_df = pd.read_csv(crime_types_file)
crime_types_df.head()

,crime_id,crime_name
0,A20,Assault and related offences
1,A50,Robbery
2,A70,"Stalking, harassment and threatening behaviour"
3,A80,Dangerous and negligent acts endangering people
4,B10,Arson


In [7]:
place_types_file = "01_Inputs/places_types.csv"
place_types_df = pd.read_csv(place_types_file)
place_types_df.head()

,place_id,place_type
0,0,liquor_store
1,1,police
2,2,park


In [8]:
lga_suburbs_file = "01_Inputs/lga_suburbs.csv"
lga_suburbs_df = pd.read_csv(lga_suburbs_file)
lga_suburbs_df.head()

,lga_id,suburb
0,1,bellfield
1,1,briar hill
2,1,bundoora
3,1,eaglemont
4,1,eltham


In [9]:
suburb_vs_crime_file = "01_Inputs/suburb_vs_crime.csv"
suburb_vs_crime_df = pd.read_csv(suburb_vs_crime_file)
suburb_vs_crime_df.head()

,lga_name,postcode,suburb,crime_id,crime_name,average_incident
0,Banyule,3079,ivanhoe,A20,Assault and related offences,43.600000
1,Banyule,3079,ivanhoe,A50,Robbery,3.000000
2,Banyule,3079,ivanhoe,A70,"Stalking, harassment and threatening behaviour",8.100000
3,Banyule,3079,ivanhoe,A80,Dangerous and negligent acts endangering people,3.333333
4,Banyule,3079,ivanhoe,B10,Arson,3.400000


In [10]:
place_type_vs_crime_file = "01_Inputs/place_type_vs_crime.csv"
place_type_vs_crime_df = pd.read_csv(place_type_vs_crime_file)
place_type_vs_crime_df.head()

,Unnamed: 0,place_id,crime_id,average_incident
0,0,0,A20,56.000000
1,1,0,A50,9.400000
2,2,0,A70,7.000000
3,3,0,A80,3.333333
4,4,0,B10,3.250000


In [11]:
place_type_vs_crime_df = place_type_vs_crime_df.drop(columns={"Unnamed: 0"})
place_type_vs_crime_df.head()

,place_id,crime_id,average_incident
0,0,A20,56.000000
1,0,A50,9.400000
2,0,A70,7.000000
3,0,A80,3.333333
4,0,B10,3.250000


In [12]:
suburb_vs_place_file = "01_Inputs/suburb_vs_places_type.csv"
suburb_vs_place_df = pd.read_csv(suburb_vs_place_file)
suburb_vs_place_df.head()

,suburb,place_id,place_type_count
0,abbotsford,0,15.0
1,abbotsford,1,1.0
2,abbotsford,2,20.0
3,aberfeldie,0,1.0
4,aberfeldie,1,0.0


### Create database connection

In [13]:
connection_string = f"postgres:{sql_key}@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{connection_string}')

In [14]:
# Confirm tables
engine.table_names()

['suburb_vs_place',
 'local_govt_areas',
 'lga_suburbs',
 'suburbs',
 'suburb_vs_crime',
 'crime',
 'place',
 'place_vs_crime']

### Load DataFrames into database

In [19]:
local_govt_areas_df.to_sql(name='local_govt_areas', con=engine, if_exists='append', index=False)

In [25]:
suburbs_df.to_sql(name='suburbs', con=engine, if_exists='append', index=False)

In [29]:
crime_types_df.to_sql(name='crime', con=engine, if_exists='append', index=False)

DataError: (psycopg2.errors.StringDataRightTruncation) value too long for type character varying(30)

[SQL: INSERT INTO crime (crime_id, crime_name) VALUES (%(crime_id)s, %(crime_name)s)]
[parameters: ({'crime_id': 'A20', 'crime_name': 'Assault and related offences'}, {'crime_id': 'A50', 'crime_name': 'Robbery'}, {'crime_id': 'A70', 'crime_name': 'Stalking, harassment and threatening behaviour'}, {'crime_id': 'A80', 'crime_name': 'Dangerous and negligent acts endangering people'}, {'crime_id': 'B10', 'crime_name': 'Arson'}, {'crime_id': 'B20', 'crime_name': 'Property damage'}, {'crime_id': 'B30', 'crime_name': 'Burglary/Break and enter'}, {'crime_id': 'B40', 'crime_name': 'Theft'}  ... displaying 10 of 25 total bound parameter sets ...  {'crime_id': 'F90', 'crime_name': 'Miscellaneous offences'}, {'crime_id': 'Other', 'crime_name': 'crimes against the person'})]
(Background on this error at: http://sqlalche.me/e/13/9h9h)

In [30]:
place_types_df.to_sql(name='place', con=engine, if_exists='append', index=False)

In [31]:
lga_suburbs_df.to_sql(name='lga_suburbs', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "lga_suburbs" violates foreign key constraint "fk_postcodelga"
DETAIL:  Key (suburb)=(st helena) is not present in table "suburbs".

[SQL: INSERT INTO lga_suburbs (lga_id, suburb) VALUES (%(lga_id)s, %(suburb)s)]
[parameters: ({'lga_id': 1, 'suburb': 'bellfield'}, {'lga_id': 1, 'suburb': 'briar hill'}, {'lga_id': 1, 'suburb': 'bundoora'}, {'lga_id': 1, 'suburb': 'eaglemont'}, {'lga_id': 1, 'suburb': 'eltham'}, {'lga_id': 1, 'suburb': 'eltham north'}, {'lga_id': 1, 'suburb': 'greensborough'}, {'lga_id': 1, 'suburb': 'heidelberg'}  ... displaying 10 of 375 total bound parameter sets ...  {'lga_id': 19, 'suburb': 'upwey'}, {'lga_id': 19, 'suburb': 'wandin north'})]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [32]:
suburb_vs_crime_df.to_sql(name='suburb_vs_crime', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "suburb_vs_crime" violates foreign key constraint "fk_crimepostcode"
DETAIL:  Key (crime_id)=(A20) is not present in table "crime".

[SQL: INSERT INTO suburb_vs_crime (lga_name, postcode, suburb, crime_id, crime_name, average_incident) VALUES (%(lga_name)s, %(postcode)s, %(suburb)s, %(crime_id)s, %(crime_name)s, %(average_incident)s)]
[parameters: ({'lga_name': 'Banyule', 'postcode': 3079, 'suburb': 'ivanhoe', 'crime_id': 'A20', 'crime_name': 'Assault and related offences', 'average_incident': 43.6}, {'lga_name': 'Banyule', 'postcode': 3079, 'suburb': 'ivanhoe', 'crime_id': 'A50', 'crime_name': 'Robbery', 'average_incident': 3.0}, {'lga_name': 'Banyule', 'postcode': 3079, 'suburb': 'ivanhoe', 'crime_id': 'A70', 'crime_name': 'Stalking, harassment and threatening behaviour', 'average_incident': 8.1}, {'lga_name': 'Banyule', 'postcode': 3079, 'suburb': 'ivanhoe', 'crime_id': 'A80', 'crime_name': 'Dangerous an

In [33]:
place_type_vs_crime_df.to_sql(name='place_vs_crime', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "place_vs_crime" violates foreign key constraint "fk_crimeplace"
DETAIL:  Key (crime_id)=(A20) is not present in table "crime".

[SQL: INSERT INTO place_vs_crime (place_id, crime_id, average_incident) VALUES (%(place_id)s, %(crime_id)s, %(average_incident)s)]
[parameters: ({'place_id': 0, 'crime_id': 'A20', 'average_incident': 56.0}, {'place_id': 0, 'crime_id': 'A50', 'average_incident': 9.4}, {'place_id': 0, 'crime_id': 'A70', 'average_incident': 7.0}, {'place_id': 0, 'crime_id': 'A80', 'average_incident': 3.333333333333333}, {'place_id': 0, 'crime_id': 'B10', 'average_incident': 3.25}, {'place_id': 0, 'crime_id': 'B20', 'average_incident': 77.6}, {'place_id': 0, 'crime_id': 'B30', 'average_incident': 112.3}, {'place_id': 0, 'crime_id': 'B40', 'average_incident': 449.3}  ... displaying 10 of 22965 total bound parameter sets ...  {'place_id': 2, 'crime_id': 'F90', 'average_incident': 8.333333333333334}, {'p

In [34]:
suburb_vs_place_type_df.to_sql(name='suburb_vs_place', con=engine, if_exists='append', index=False)

NameError: name 'suburb_vs_place_type_df' is not defined